### 참고 url
https://sy-log.tistory.com/entry/%EC%85%80%EB%A0%88%EB%8B%88%EC%9B%80-%ED%81%AC%EB%A1%A4%EB%A7%81-%EC%98%88%EC%A0%9C-%EB%8B%A4%EB%82%98%EC%99%80-%EB%A6%AC%EB%B7%B0-%ED%81%AC%EB%A1%A4%EB%A7%81%EC%9D%84-%EC%9C%84%ED%95%9C-%EC%A7%80%EC%86%8D%EA%B0%80%EB%8A%A5%ED%95%9C-%EC%BD%94%EB%93%9C-%EB%A7%8C%EB%93%A4%EA%B8%B0-1

# 1-0. 카테고리 url 접속

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup
import time
import math
import pandas as pd
import pickle

options = Options()
options.headless = True  # 브라우저를 열지 않음
options.add_argument("--window-size=1920x1080")
options.add_argument("disable-gpu")

# 크롬 드라이버 경로 설정
driver_path = '/path/to/chromedriver'  # 실제 chromedriver 경로로 수정

driver = webdriver.Chrome()


url = "https://prod.danawa.com/list/?cate=10352481&searchOption=/innerSearchKeyword="
driver.get(url)

C:\Users\JK Kwak\AppData\Local\Temp\ipykernel_26936\1291539210.py:15: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True  # 브라우저를 열지 않음


# 1-1. 인기 3사만 필터링(옵션)

In [2]:
temp = driver.find_elements(By.XPATH, '//*[@id="dlMaker_simple"]/dd/ul[1]/li')

print(len(temp))

for i in temp:
    print(i.text)

5
LG전자
삼성전자
위닉스
샤오미
쿠쿠전자


In [3]:
for i in temp:
    if 'LG전자' in i.text:
        check_lg = i
    elif '삼성전자' in i.text:
        check_ss = i
    elif '위닉스' in i.text:
        check_wn = i
         

In [4]:
check_lg.click()
check_ss.click()
check_wn.click()

#무엇을 클릭하냐에 따라 필터링

In [5]:
def manfacturer_checkbox(target_list):
    
    #제조사별 부분 XPATH 리스트로 가져오기
    manfacturer_list = driver.find_elements(By.XPATH, '//*[@id="dlMaker_simple"]/dd/ul[1]/li')
    
    #타겟 제조사 체크박스 하나씩 클릭
    for target in target_list:
        for m in manufacturer_list:
            if target in m.text:
                m.click()
                
                break
                
    print(str(target_list), "선택 완료")

# 1-2. 페이지 이동하며 제품정보로 이동

In [6]:
#몇 페이지인지 알기
#모델 개수 
num_models = driver.find_element(By.CLASS_NAME, 'list_num').text.strip().replace('(','').replace(')','')

#페이지 개수(페이지 당 보이는 제품은 30개)
num_pages = math.ceil(int(num_models)/30)

print(num_models)
print(num_pages)

388
13


In [7]:
#페이지 변수 생성, 시작은 1페이지부터
page = 1

#페이지에 1씩 더하면서, 전체 페이지 수와 같아지기 전까지 반복
while page < num_pages:
    
    page += 1
    
    ### 페이지 크롤링 코드는 여기에 추가 ###
    
    #11,21,31 페이지인 경우 [다음 페이지]버튼 클릭
    
    if page%10 ==1:
        driver.find_element(By.CSS_SELECTOR, '#productListArea > div.prod_num_nav > div > a').click()
        # Copy selector로 가져옴
        
    #나머지는 page에 해당하는 페이지 클릭
    else: 
        #페이지 넘기는 부분 경로 통째로 가져와서
        page_nums = driver.find_element(By.XPATH, '//*[@id="productListArea"]/div[4]/div/div')
        #다음 페이지 링크 클릭
        page_nums.find_element(By.LINK_TEXT, str(page)).click()
        
    #에러 방지 페이지마다 잠시 대기
    time.sleep(2)


# 1-3 제품 정보 dataframe에 저장

## 1-3-1 가격정보

In [7]:
#이 페이지 전부 긁어오기
soup = BeautifulSoup(driver.page_source)

#제품 정보들을 전부 가져오기
products = soup.select('div.prod_main_info')

#가격정보 정보 리스트로 가져오기
pricelist = products[0].select('div.prod_pricelist li') #왜 li이지?

#1번째 제품의 1번째 가격 정보
pricelist[0].select_one('div.over_preview').text.strip()

'일반구매'

In [8]:
#1번째 제품의 1번째 가격 (원)
pricelist[0].select_one('p.price_sect a').text.strip() # a가 왜 붙지?

'736,090원'

In [9]:
#가격정보 저장 리스트
numofprices_list = []

#페이지별 소스 저장 딕셔너리 
soup_dict = {}

page = 1

#page에 1씩 더하면서 전체 페이지 수와 같아지기 전까지 반복
while page < num_pages:
    
    #제품 정보들 전부 가져오기
    soup = BeautifulSoup(driver.page_source)
    products = soup.select('div.prod_main_info')
    time.sleep(1)
    
    #페이지별 소스 딕셔너리에 저장
    soup_dict[page] = products
    
    #제품 하나씩 돌리면서
    for i in range(len(products)):
        try: #가격 정보 개수 가졍괴
            numofprices_list.append(len(products[i].select('div.prod_pricelist li')))
        except:
            numofprices_list.append('')
            
    time.sleep(1)
    print(page, "페이지 완료")
    
    page +=1
    
    #11,21,31 번째 페이지로 넘기는 경우 [화살표] 버튼 클릭
    if page%10 ==1:
        driver.find_element(By.CSS_SELECTOR, '#productListArea > div.prod_num_nav > div > a').click()
    
    #나머지는 page에 해당하는 페이지 클릭
    else: 
        #페이지 넘기는 부분 경로 통째로 가져와서
        page_nums = driver.find_element(By.XPATH, '//*[@id="productListArea"]/div[4]/div/div')
        #다음 페이지 링크 클릭
        page_nums.find_element(By.LINK_TEXT, str(page)).click()
        
    #에러 방지 페이지마다 잠시 대기
    time.sleep(2)
    

#마지막 페이지 소스 가져오고 마무리
soup = BeautifulSoup(driver.page_source)
products = soup.select('div.prod_main_info')
soup_dict[page] = products
print(page, '페이지 완료')

print('<<<크롤링 완료입니다>>>')
print('가격정보가 가장 많은 경우는', max(numofprices_list), '개 입니다!')

1 페이지 완료
2 페이지 완료
3 페이지 완료
4 페이지 완료
5 페이지 완료
6 페이지 완료
7 페이지 완료


KeyboardInterrupt: 

## 1-3-2. 저장해둔 페이지 소스에서 쓸만한 제품정보만 가져와서 DataFrame으로 합

In [ ]:
def crawling_danawa(soup_dict, max_numofprices_list):
    
    #모델이름, 출시시기, 헤드메시지, 스펙, 리뷰평점, 리뷰개수, 페이지링크 가져올 빈 리스트
    name_list, hm_list, spec_list, release_list, star_list, review_list, link_list = [], [], [], [], [], [], []
    
    #가격정보 리스트셋 (ex. 1위 일반구매, 2위 추가필터 포함...)
    priceinfo_list_set = []
    
    #최저가 리스트셋 (ex. 100,000원, 120,000원)
    price_list_set = []
    
    #최댓값만큼 빈 리스트 생성(이중리스트)
    for i in range(max_numofprices_list):
        priceinfo_list_set.append([])
        price_list_set.append([])
        
    #딕셔너리에 저장해둔 페이지 하나씩 돌리기
    for key in range(len(soup_dict)):
        products = soup_dict[key+1]
        
        #제품마다 하나씩 돌리기
        for i in range(len(products)):
            
            #모델 이름
            try:
                name_list.append(products[i].select_one('p.prod_name a').text.strip())
            except:
                name_list.append('')
                
            #모델 출시시기
            try:
                release_list.append(products[i].select_one('div.prod_sub_info dd').text.strip()) #div 뭉텅이 안의 dd 태그에 날짜 있음
            except:
                release_list.append('')
                
            #헤드 메시지
            try:
                hm_list.append(products[i].select_one('div.prod_intro').text.strip()) 
            except:
                hm_list.append('')
                
            #모델 스펙
            try:
                spec_list.append(products[i].select_one('div.spec_list').text.strip()) 
            except:
                spec_list.append('')
                
            #리뷰 평점 & 개수
            try:
                star_list.append(products[i].select_one('div.cnt_star').text.strip().replace('점',''))
                review_list.append(products[i].select_one('div.prod_sub_info a').text.strip())
            except:
                star_list.append('')
                review_list.append('')
                
            #제품페이지 링크
            try:
                thumbnail = products[i].select_one('div.thumb_image a')
                link_list.append(thumbnail['href'])
            except:
                link_list.append('')
                
            #가격 정보
            pricelist_raw = products[i].select('div.prod_pricelist li')
            
            for p in range(max_numofprices_list):
                try:
                    priceinfo_list_set[p].append(
                        pricelist_raw[p].select_one('div.over_preview').text.strip())
                except:
                    priceinfo_list_set[p].append('')
                    
            for p in range(max_numofprices_list):
                try:
                    price_list_set[p].append(pricelist_raw[p].select_one('p.price_sect a').text.strip())
                except:
                    price_list_set[p].append('')
     
    #DataFrame 저장
    df_result = pd.DataFrame({
        '제품모델명' : name_list, '출시시기' : release_list,
        '리뷰평점' : star_list, '리뷰개수' : review_list,
        '헤드메시지': hm_list, '스펙' : spec_list, '링크 url': link_list
    })
    
    for p in range(max_numofprices_list):
        df_result['가격정보_{}'.format(str(p+1))] = priceinfo_list_set[p]
        df_result['최저가_{}'.format(str(p+1))] = price_list_set[p]
        
    return df_result
    

In [13]:
df_test = crawling_danawa(soup_dict, max(numofprices_list)); df_test

,제품모델명,출시시기,리뷰평점,리뷰개수,헤드메시지,스펙,링크 url,가격정보_1,최저가_1,가격정보_2,최저가_2,가격정보_3,최저가_3
0,LG전자 퓨리케어 360˚ 플러스 AS303DWFA,2023.01.,4.8,152,이제 플러스 모델에도 UP가전으로 업그레이드 가능!,공기청정기 / 30.3평(100㎡) / 소비전력: 70W / 버튼잠금 / 풍량조절 ...,https://prod.danawa.com/info/?pcode=18775037&c...,일반구매,"746,160원",무빙휠 포함,"858,980원",,
1,삼성전자 블루스카이 3100 AX033B310GGD,2022.03.,4.7,"1,065",,공기청정기 / 10평(33㎡) / 소비전력: 26W / 버튼잠금 / 풍량조절 / 스...,https://prod.danawa.com/info/?pcode=16536392&c...,1개,"153,000원",2개,"382,700원",추가필터 포함,"269,000원"
2,삼성전자 블루스카이 5500 AX060CG500GGD,2023.01.,4.6,91,실내 공기질까지 학습해 맞춰주는 AI 청정기능 적용,공기청정기 / 18평(60㎡) / 소비전력: 60W / 버튼잠금 / 풍량조절 / 이...,https://prod.danawa.com/info/?pcode=18852014&c...,일반구매,"247,690원",추가필터 포함,"419,000원",,
3,LG전자 오브제컬렉션 퓨리케어 360˚ UV살균 AS354NS3A,2024.02.,4.9,68,LG전자 24년형 PA인증받은 UV살균 공기청정기 출시!,공기청정기 / 34평(114㎡) / 소비전력: 80W / 버튼잠금 / 풍량조절 / ...,https://prod.danawa.com/info/?pcode=34738715&c...,일반구매,"1,313,620원",무빙휠 포함,"1,423,860원",,
4,LG전자 퓨리케어 360˚ Hit AS153HWWC,2023.03.,4.8,678,퓨리케어360˚ 1단 모델 대비 16% 슬림해진 에어라운드 디자인 적용,공기청정기 / 15평(50㎡) / 소비전력: 34W / 버튼잠금 / 풍량조절 / 스...,https://prod.danawa.com/info/?pcode=19312541&c...,,"311,380원",,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
396,삼성전자 무풍 큐브 AX67R9980BFD,2019.02.,4.6,18,,공기청정기 / 20.2평(67㎡) / 소비전력: 47W / 버튼잠금 / 이동식바퀴 ...,https://prod.danawa.com/info/?pcode=7196452&ca...,,"976,000원",,,,
397,삼성전자 무풍 큐브 AX94N9980SSD,2018.02.,4.5,71,,공기청정기 / 28.4평(94㎡) / 소비전력: 70W / 스마트기능: 삼성 Sma...,https://prod.danawa.com/info/?pcode=5908541&ca...,,"1,109,900원",,,,
398,삼성전자 블루스카이 5000 AX50M5050WDD,2017.01.,4.6,89,,공기청정기 / 15평(50㎡) / 소비전력: 35W / 버튼잠금 / 이동식바퀴 / ...,https://prod.danawa.com/info/?pcode=4837558&ca...,,"499,000원",,,,
399,삼성전자 블루스카이 3000 AX40K3021UWD,2016.01.,4.7,88,,공기청정기 / 11.9평(39㎡) / 소비전력: 34W / 버튼잠금 / 스마트기능:...,https://prod.danawa.com/info/?pcode=3660984&ca...,,"399,000원",,,,


# 1-4 문제 없나 확인

In [ ]:
#중복 데이터 확인 (388개 < 401개)
dup = df_test.duplicated(['제품모델명']); dup

In [15]:
#중복 행 개수 확인
dnum = 0 
for d in dup:
    if d == True:
        dnum += 1
        
dnum

0